In [9]:
import requests # Для работы с POST и GET
import json # Для записи в файл JSON
import copy # Для deepcopy
import time # Для текущего времени
import os.path # Для проверки существования файла

urlAmo = 'https://testcallapi.amocrm.ru'

urlAuthAmo = '/private/api/auth.php?type=json'
urlPipeAmo = '/api/v2/pipelines'
urlLeadAmo = '/api/v2/leads'

# Все данные для работы

headers = {'Content-Type': 'application/json'}

# Какой ID этапа для автообзвона
leadsStatusId = 32121847 

# В какой ID этапа переносим при дозвоне
leadsStatusIdOk = 143

# В какой отдел направлять вызов
sipUri = '726@test.gravitel.ru' 

# Cо скольки и до скольки автообзвон
timeFrom = '19:00'
timeTo = '20:00'

# ТОКЕН Call API
tokenCallApi = '232c7a1e8be4f3e5bded4d7fa848018f' 

# Логин админа в АМО СРМ
loginUserAmo = 'nd@gravitel.ru' 

# Hash этого сотрудника
hashUserAmo = '5e0db73b6bb4ddf46fcb4107090cffdcc5026961' 

# Какой ТЭГ проставляем при недозвоне

tagLead = 'НЕДОЗВОН'

# Какой ТЭГ проставляем, если 3 раза не дозвонились.


# Setting parameters

authKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}

# Connect with amoCRM POST

authAmo = requests.post(urlAmo + urlAuthAmo, headers=headers, json=authKeyAmo).json()

# check login & Hash

if 'error_code' in authAmo['response'].keys():
    print(authAmo['response']['error'])
    
# Получить все сделки, метод GET

leadKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}
leads = requests.get(urlAmo + urlLeadAmo, headers=headers, params=leadKeyAmo).json()

# Проверка на ошибки

if 'title' in leads.keys():
    print(leads['detail'])

# Делаем из словаря список словарей
    
leads = leads['_embedded']['items']

# Удаляем сделки, если этап не leadsStatusId

for i in reversed(range(len(leads))):
    if leads[i]['status_id'] != leadsStatusId:
        del leads[i]

# Удаляем лишние ключи

leadsClear = copy.deepcopy(leads)

for i, elem in enumerate(leads):
    for key in leads[i]:
        if key not in ('id', 'main_contact'):
            leadsClear[i].pop(key)
            
            
# Проверяем есть ли контакт у сделки, если нет, то удаляем сделку из БД.

for i in reversed(range(len(leadsClear))):
    if leadsClear[i]['main_contact'] == {}: 
        del leadsClear[i]

        
# Достаем номер из Контакта сделки

for elem in leadsClear:
    urlContactAmo = elem['main_contact']['_links']['self']['href']
    contact = requests.get(urlAmo + urlContactAmo, headers=headers, params=leadKeyAmo).json()
    elem['main_contact'] = int(contact['_embedded']['items'][0]['custom_fields'][0]['values'][0]['value'])
    
    # Добавляем счетчик количества обзвонов, если его нет

    if 'count' not in elem:
        elem['count'] = 3


# Сверяем с DB_dialouts.json
if os.path.isfile('DB_dialouts.json') == True:
    with open("DB_dialouts.json", "r") as dialoutsFile:
        dbForCheck = json.load(dialoutsFile)
    dialoutsFile.close()
    for i in reversed(range(len(leadsClear))):
        for j in reversed(range(len(dbForCheck))):
            if leadsClear[i]['id'] == dbForCheck[j]['id']:
                del leadsClear[i]
    leadsClear.extend(dbForCheck)
leadsClear

IndexError: list index out of range

In [31]:
# Парсим номера для автообзвона

numbers = [x['main_contact'] for x in leadsClear]

# Создаем автообзвон

urlCreateDialOuts = 'https://callapi.gravitel.ru/api/v1/createdialout'
jsonCreateCallApi = {'connectaction': '2', 
                     'linecnt': '1', 
                     'numbers': numbers, 
                     'redirecturi': sipUri, 
                     'timefrom': timeFrom, 
                     'timeto': timeTo, 
                     'token': tokenCallApi, 
                     'type': '1'}
createDialOuts = requests.post(urlCreateDialOuts, headers=headers, json=jsonCreateCallApi).json()
if createDialOuts['status'] == 'error':
    print ('Error code:', createDialOuts['errorCode'])
    print ('Error text:', createDialOuts['errorText'])
    #exit()
else:
    dialoutid = createDialOuts['result']['dialoutid']

    # Записываем в файл id автообзвона

    dialoutidFile = open('dialoutid.txt', 'w')
    dialoutidFile.write(str(dialoutid))
    dialoutidFile.close()
createDialOuts

[74952301001, 79096546816, 79852721228, 1234]
[{'id': 2036873, 'main_contact': 74952301001, 'count': 3}, {'id': 2036907, 'main_contact': 79096546816, 'count': 3}, {'id': 2039071, 'main_contact': 79852721228, 'count': 3}, {'id': 2045945, 'main_contact': 1234, 'count': 3}]


{'status': 'success', 'result': {'dialoutid': 7570}}

In [6]:
fileDialout = open('dialoutid.txt', 'r')
dialoutid = int(fileDialout.read())
print(type(dialoutid))
fileDialout.close()

<class 'int'>


In [3]:
# Получить все воронки и этапы, метод GET
'''
pipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}
pipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()

if 'title' in pipelines.keys():
    print(pipelines['detail'])


'''

"\npipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}\npipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()\n\nif 'title' in pipelines.keys():\n    print(pipelines['detail'])\n\n\n"

# Для ДЕБАГА

In [79]:
try:
    import http.client as http_client
except ImportError:
    # Python 2
    import httplib as http_client
http_client.HTTPConnection.debuglevel = 1

# You must initialize logging, otherwise you'll not see debug output.
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True


# Test
import requests
numbers = [x['main_contact'] for x in leadsClear]
urlCreateDialOuts = 'https://callapi.gravitel.ru/api/v1/createdialout'
headersCallApi = {'Content-Type': 'application/json'}
jsonCreateCallApi = {}
rs = requests.post(urlCreateDialOuts, headers=headersCallApi, json=jsonCreateCallApi)
print(rs)
print(rs.url)
print(rs.history)
print(rs.cookies)
print(rs.text)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): callapi.gravitel.ru:443
DEBUG:urllib3.connectionpool:https://callapi.gravitel.ru:443 "POST /api/v1/createdialout HTTP/1.1" 400 62


send: b'POST /api/v1/createdialout HTTP/1.1\r\nHost: callapi.gravitel.ru\r\nUser-Agent: python-requests/2.22.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nContent-Length: 264\r\n\r\n'
send: b'{"connectaction": "2", "linecnt": "1", "numbers": [79852721228, 79096546816, 79852721228, 74952302222], "redirecturi": "100@test.gravitel.ru", "timefrom": "12:00", "timeto": "16:00", "token": "232c7a1e8be4f3e5bded4d7fa848018f", "type": "1", "userdata": "12345, 3"}'
reply: 'HTTP/1.1 400 Bad Request\r\n'
header: Server: nginx/1.13.5
header: Date: Mon, 16 Mar 2020 11:03:30 GMT
header: Content-Type: application/json
header: Content-Length: 62
header: Connection: keep-alive
header: X-Powered-By: GravitelAPI v1
header: Last-Modified: Mon, 16 Mar 2020 11:03:30 GMT
header: Access-Control-Allow-Origin: *
header: Access-Control-Allow-Methods: GET,POST,PUT,DELETE
<Response [400]>
https://callapi.gravitel.ru/api/v1/createdialout
[]
<RequestsC

In [8]:
# Обработка автообзвона 

dialoutid = 7384
urlGetDialOutStat = 'https://callapi.gravitel.ru/api/v1/getdialoutstat'

jsonGetDialOutStat = {'token': tokenCallApi,
                      'dialoutid': dialoutid}

# Получаем статистику
getDialOutStat = requests.post(urlGetDialOutStat, headers=headers, json=jsonGetDialOutStat).json()
getDialOutStat = getDialOutStat['result']['data']

# Добавляем статус в БД
for i, elem in enumerate(getDialOutStat):
    for j in enumerate(leadsClear):
        if getDialOutStat[i]['number'] == leadsClear[j]['main_contact']:
            leadsClear[j]['status'] = getDialOutStat[i]['intstatus']
            
leadsClear

KeyError: 'result'

In [154]:
# Проставляем Тэги если НЕДОЗВОН

# Connect with amoCRM POST
for i, elem in enumerate(leadsClear):
    
    # Ставим ТЭГ tagLead, если недозвон, то есть статус не 2
    if leadsClear[i]['status'] != 2:
        leadsClear[i]['count'] = leadsClear[i]['count'] - 1
        jsonUpdateLead = {'USER_LOGIN': loginUserAmo, 
                          'USER_HASH': hashUserAmo,
                          'update':
                                  [{'id': leadsClear[i]['id'], 
                                    'tags': tagLead,
                                    'updated_at': time.time()}]}
        answerUpdateLeads = requests.post(urlAmo + urlLeadAmo, headers=headers, json=jsonUpdateLead).json()
        
        
    # Перекидываем сделку на другой этап (id = leadsStatusIdOk), если было 3 недозвона
    if leadsClear[i]['count'] == 0:
        jsonUpdateLead = {'USER_LOGIN': loginUserAmo, 
                          'USER_HASH': hashUserAmo,
                          'update':
                                  [{'id': leadsClear[i]['id'],
                                    'status_id': leadsStatusIdOk,
                                    'updated_at': time.time()}]}
        answerUpdateLeads = requests.post(urlAmo + urlLeadAmo, headers=headers, json=jsonUpdateLead).json()
leadsClear

[{'id': 2039071, 'main_contact': 79852721228, 'count': 1, 'status': 5},
 {'id': 2036873, 'main_contact': 74952301001, 'count': 3, 'status': 2},
 {'id': 2036907, 'main_contact': 79096546816, 'count': 3, 'status': 2}]

In [155]:
# Убираем сделку, если count = 0 и если был дозвон


for i in reversed(range(len(leadsClear))):
    if leadsClear[i]['count'] == 0 or leadsClear[i]['status'] == 2:
        del leadsClear[i]
leadsClear

[{'id': 2039071, 'main_contact': 79852721228, 'count': 1, 'status': 5}]

In [156]:
# Запись в файл

json.dump(leadsClear,open("DB_dialouts.json","w"))

In [44]:

time.time()

1584442944.0534003

In [45]:
'''
for i, elem in enumerate(leads):
    for leadsClear_values in iter(leadsClear.items()):
        if leads[i]['responsible_user_id'] == leadsClear[leadsClear_values]: 
            print ('Ok', 'responsible_user_id', leadsClear_values)
        else: 
            print ('Not', leads_values, leadsClear_values)
'''

"\nfor i, elem in enumerate(leads):\n    for leadsClear_values in iter(leadsClear.items()):\n        if leads[i]['responsible_user_id'] == leadsClear[leadsClear_values]: \n            print ('Ok', 'responsible_user_id', leadsClear_values)\n        else: \n            print ('Not', leads_values, leadsClear_values)\n"